# 卷积神经网络基础实践：MNIST手写数字识别

## 概述

本教程实现一个经典的卷积神经网络(Convolutional Neural Network, CNN)，用于MNIST手写数字识别任务。
通过本教程，你将深入理解CNN的核心组件及其工作原理。

## 知识要点

1. **卷积层(Conv2D)**：提取图像的局部特征，通过可学习的滤波器在输入上滑动进行特征提取
2. **池化层(MaxPooling2D)**：对特征图进行下采样，减少参数数量并增强平移不变性
3. **全连接层(Dense)**：将提取的特征映射到最终的分类结果
4. **激活函数**：ReLU用于隐藏层引入非线性，Softmax用于输出层生成概率分布

## 网络架构

```
输入层 (28×28×1)
    ↓
Conv2D (32 filters, 3×3) → ReLU → 输出: 26×26×32
    ↓
MaxPooling2D (2×2) → 输出: 13×13×32
    ↓
Conv2D (64 filters, 3×3) → ReLU → 输出: 11×11×64
    ↓
MaxPooling2D (2×2) → 输出: 5×5×64
    ↓
Conv2D (64 filters, 3×3) → ReLU → 输出: 3×3×64
    ↓
Flatten → 输出: 576
    ↓
Dense (64 units) → ReLU
    ↓
Dense (10 units) → Softmax → 输出: 10类概率
```

## 1. 环境配置与依赖导入

In [ ]:
# -*- coding: utf-8 -*-
"""
卷积神经网络基础实现
使用Keras构建CNN进行MNIST手写数字识别
"""

import os
import warnings

# 抑制TensorFlow信息日志
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt

# Keras组件导入
from keras import layers, models
from keras.datasets import mnist
from keras.utils import to_categorical

# 设置随机种子，确保结果可复现
np.random.seed(42)

# 中文显示配置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境配置完成")

## 2. 数据加载与预处理

MNIST数据集包含70,000张28×28像素的灰度手写数字图像：
- 训练集：60,000张
- 测试集：10,000张
- 类别：0-9共10个数字

### 预处理步骤
1. **维度调整**：将图像从(28,28)调整为(28,28,1)以适配卷积层输入要求
2. **归一化**：将像素值从[0,255]缩放到[0,1]，加速收敛并提升稳定性
3. **One-Hot编码**：将标签转换为向量形式，适配Softmax输出

In [ ]:
def load_and_preprocess_data():
    """
    加载并预处理MNIST数据集
    
    Returns:
        tuple: (train_images, train_labels, test_images, test_labels)
               预处理后的训练和测试数据
    """
    # 加载原始数据
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
    
    print(f"原始训练集形状: {train_images.shape}")
    print(f"原始测试集形状: {test_images.shape}")
    
    # 维度调整：添加通道维度
    # 卷积层期望输入形状为 (batch_size, height, width, channels)
    train_images = train_images.reshape((60000, 28, 28, 1))
    test_images = test_images.reshape((10000, 28, 28, 1))
    
    # 归一化：将像素值缩放到[0,1]区间
    # 这有助于梯度下降更快收敛
    train_images = train_images.astype('float32') / 255.0
    test_images = test_images.astype('float32') / 255.0
    
    # One-Hot编码
    # 将整数标签转换为二进制类别矩阵
    # 例如：3 → [0,0,0,1,0,0,0,0,0,0]
    train_labels = to_categorical(train_labels, num_classes=10)
    test_labels = to_categorical(test_labels, num_classes=10)
    
    print(f"处理后训练集形状: {train_images.shape}")
    print(f"标签形状: {train_labels.shape}")
    
    return train_images, train_labels, test_images, test_labels


# 执行数据加载
train_images, train_labels, test_images, test_labels = load_and_preprocess_data()

### 数据可视化

查看数据集中的样本图像，直观理解数据特征。

In [ ]:
def visualize_samples(images, labels, num_samples=10):
    """
    可视化数据样本
    
    Args:
        images: 图像数组
        labels: One-Hot编码的标签
        num_samples: 显示的样本数量
    """
    fig, axes = plt.subplots(2, 5, figsize=(12, 5))
    axes = axes.flatten()
    
    # 随机选择样本索引
    indices = np.random.choice(len(images), num_samples, replace=False)
    
    for ax, idx in zip(axes, indices):
        # 显示图像，squeeze去除单通道维度
        ax.imshow(images[idx].squeeze(), cmap='gray')
        # 从One-Hot编码恢复原始标签
        label = np.argmax(labels[idx])
        ax.set_title(f'Label: {label}')
        ax.axis('off')
    
    plt.suptitle('MNIST Sample Images', fontsize=14)
    plt.tight_layout()
    plt.show()


# 展示训练集样本
visualize_samples(train_images, train_labels)

## 3. 构建卷积神经网络

### 3.1 卷积层原理

卷积操作通过滤波器(kernel)在输入图像上滑动，计算局部区域的加权和：

**输出尺寸计算公式**（无padding时）：
$$output\_size = input\_size - kernel\_size + 1$$

例如：输入28×28，3×3卷积核 → 输出26×26

### 3.2 池化层原理

最大池化选取窗口内的最大值，实现特征降维：
- 减少计算量和参数数量
- 增强特征的平移不变性
- 扩大后续层的感受野

In [ ]:
def build_cnn_model(input_shape=(28, 28, 1), num_classes=10):
    """
    构建卷积神经网络模型
    
    Args:
        input_shape: 输入图像形状，默认(28, 28, 1)
        num_classes: 分类类别数，默认10
    
    Returns:
        model: 编译好的Keras模型
    """
    model = models.Sequential(name='SimpleCNN')
    
    # ========== 特征提取部分 ==========
    
    # 第一个卷积块
    # 32个3×3滤波器，提取低级特征（边缘、角点等）
    model.add(layers.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        activation='relu',
        input_shape=input_shape,
        name='conv1'
    ))
    # 2×2最大池化，将特征图尺寸减半
    model.add(layers.MaxPooling2D(pool_size=(2, 2), name='pool1'))
    
    # 第二个卷积块
    # 64个滤波器，提取中级特征（纹理、局部形状）
    model.add(layers.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        activation='relu',
        name='conv2'
    ))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), name='pool2'))
    
    # 第三个卷积块
    # 继续提取高级特征（数字的整体形状）
    model.add(layers.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        activation='relu',
        name='conv3'
    ))
    
    # ========== 分类器部分 ==========
    
    # 展平层：将3D特征图转换为1D向量
    # 3×3×64 = 576个特征
    model.add(layers.Flatten(name='flatten'))
    
    # 全连接层：学习特征组合
    model.add(layers.Dense(64, activation='relu', name='fc1'))
    
    # 输出层：Softmax激活产生概率分布
    model.add(layers.Dense(num_classes, activation='softmax', name='output'))
    
    return model


# 构建模型
model = build_cnn_model()

# 查看模型架构
print("模型架构概览：")
print("=" * 65)
model.summary()

### 3.3 参数量分析

理解每层的参数计算对于模型设计至关重要：

| 层 | 参数计算 | 参数量 |
|---|---|---|
| conv1 | (3×3×1+1)×32 | 320 |
| conv2 | (3×3×32+1)×64 | 18,496 |
| conv3 | (3×3×64+1)×64 | 36,928 |
| fc1 | (576+1)×64 | 36,928 |
| output | (64+1)×10 | 650 |
| **总计** | | **93,322** |

## 4. 模型编译与训练

### 训练配置

- **优化器(Adam)**：自适应学习率优化算法，结合了Momentum和RMSprop的优点
- **损失函数(Categorical Crossentropy)**：多分类任务的标准损失函数
- **评估指标(Accuracy)**：分类正确率

In [ ]:
# 编译模型
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("模型编译完成")
print(f"优化器: Adam")
print(f"损失函数: Categorical Crossentropy")
print(f"评估指标: Accuracy")

In [ ]:
# 训练参数配置
EPOCHS = 5          # 训练轮次
BATCH_SIZE = 64     # 批次大小
VALIDATION_SPLIT = 0.1  # 验证集比例

print(f"开始训练...")
print(f"训练轮次: {EPOCHS}")
print(f"批次大小: {BATCH_SIZE}")
print(f"验证集比例: {VALIDATION_SPLIT * 100}%")
print("=" * 50)

# 训练模型
history = model.fit(
    train_images,
    train_labels,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    verbose=1
)

print("\n训练完成！")

### 训练历史可视化

通过可视化训练曲线，可以判断模型是否存在过拟合或欠拟合问题。

In [ ]:
def plot_training_history(history):
    """
    绘制训练历史曲线
    
    Args:
        history: model.fit()返回的History对象
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # 准确率曲线
    axes[0].plot(history.history['accuracy'], label='Training', marker='o')
    axes[0].plot(history.history['val_accuracy'], label='Validation', marker='s')
    axes[0].set_title('Model Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 损失曲线
    axes[1].plot(history.history['loss'], label='Training', marker='o')
    axes[1].plot(history.history['val_loss'], label='Validation', marker='s')
    axes[1].set_title('Model Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


# 绘制训练历史
plot_training_history(history)

## 5. 模型评估

在测试集上评估模型的泛化性能，测试集数据在训练过程中从未被模型见过。

In [ ]:
# 测试集评估
print("在测试集上评估模型...")
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=1)

print("\n" + "=" * 40)
print("测试集评估结果")
print("=" * 40)
print(f"测试损失: {test_loss:.4f}")
print(f"测试准确率: {test_accuracy:.4f} ({test_accuracy * 100:.2f}%)")

### 预测结果可视化

查看模型在测试样本上的预测结果，包括正确和错误的预测案例。

In [ ]:
def visualize_predictions(model, images, labels, num_samples=10):
    """
    可视化模型预测结果
    
    Args:
        model: 训练好的模型
        images: 测试图像
        labels: 真实标签(One-Hot)
        num_samples: 显示样本数量
    """
    # 随机选择样本
    indices = np.random.choice(len(images), num_samples, replace=False)
    sample_images = images[indices]
    sample_labels = labels[indices]
    
    # 进行预测
    predictions = model.predict(sample_images, verbose=0)
    
    # 可视化
    fig, axes = plt.subplots(2, 5, figsize=(14, 6))
    axes = axes.flatten()
    
    for ax, img, true_label, pred in zip(axes, sample_images, sample_labels, predictions):
        true_class = np.argmax(true_label)
        pred_class = np.argmax(pred)
        confidence = pred[pred_class] * 100
        
        ax.imshow(img.squeeze(), cmap='gray')
        
        # 根据预测正确与否设置标题颜色
        color = 'green' if true_class == pred_class else 'red'
        ax.set_title(
            f'True: {true_class}\nPred: {pred_class} ({confidence:.1f}%)',
            color=color,
            fontsize=10
        )
        ax.axis('off')
    
    plt.suptitle('Model Predictions (Green=Correct, Red=Wrong)', fontsize=12)
    plt.tight_layout()
    plt.show()


# 展示预测结果
visualize_predictions(model, test_images, test_labels)

### 混淆矩阵分析

混淆矩阵能够直观展示模型在各类别上的分类表现，帮助识别容易混淆的数字对。

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(model, images, labels):
    """
    绘制混淆矩阵
    
    Args:
        model: 训练好的模型
        images: 测试图像
        labels: 真实标签(One-Hot)
    """
    # 获取预测结果
    predictions = model.predict(images, verbose=0)
    pred_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(labels, axis=1)
    
    # 计算混淆矩阵
    cm = confusion_matrix(true_classes, pred_classes)
    
    # 绘制热力图
    plt.figure(figsize=(10, 8))
    plt.imshow(cm, interpolation='nearest', cmap='Blues')
    plt.title('Confusion Matrix', fontsize=14)
    plt.colorbar()
    
    # 添加数值标注
    thresh = cm.max() / 2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                     ha='center', va='center',
                     color='white' if cm[i, j] > thresh else 'black',
                     fontsize=9)
    
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)
    plt.xticks(range(10))
    plt.yticks(range(10))
    plt.tight_layout()
    plt.show()
    
    # 计算每类准确率
    print("\n各类别准确率:")
    for i in range(10):
        class_total = cm[i].sum()
        class_correct = cm[i, i]
        accuracy = class_correct / class_total * 100
        print(f"  数字 {i}: {accuracy:.2f}% ({class_correct}/{class_total})")


# 绘制混淆矩阵
plot_confusion_matrix(model, test_images, test_labels)

## 6. 特征图可视化

可视化卷积层的特征图，理解CNN如何逐层提取和抽象图像特征。

In [ ]:
def visualize_feature_maps(model, image, layer_names=None):
    """
    可视化指定层的特征图
    
    Args:
        model: 训练好的模型
        image: 单张输入图像 (28, 28, 1)
        layer_names: 要可视化的层名称列表
    """
    if layer_names is None:
        layer_names = ['conv1', 'conv2', 'conv3']
    
    # 构建特征提取模型（兼容Keras 2.x和3.x）
    # 使用layers.Input显式定义输入
    input_layer = layers.Input(shape=(28, 28, 1))
    x = input_layer
    layer_outputs = []
    
    for layer in model.layers:
        x = layer(x)
        if layer.name in layer_names:
            layer_outputs.append(x)
    
    feature_model = models.Model(inputs=input_layer, outputs=layer_outputs)
    
    # 获取特征图
    image_batch = np.expand_dims(image, axis=0)
    feature_maps = feature_model.predict(image_batch, verbose=0)
    
    # 确保feature_maps是列表
    if not isinstance(feature_maps, list):
        feature_maps = [feature_maps]
    
    # 可视化每层的特征图
    for layer_name, fmap in zip(layer_names, feature_maps):
        n_features = min(fmap.shape[-1], 16)  # 最多显示16个特征图
        
        fig, axes = plt.subplots(2, 8, figsize=(14, 4))
        axes = axes.flatten()
        
        for i in range(n_features):
            axes[i].imshow(fmap[0, :, :, i], cmap='viridis')
            axes[i].axis('off')
        
        # 隐藏多余的子图
        for i in range(n_features, len(axes)):
            axes[i].axis('off')
        
        plt.suptitle(f'Feature Maps - {layer_name} ({fmap.shape[1]}x{fmap.shape[2]}x{fmap.shape[3]})', 
                     fontsize=12)
        plt.tight_layout()
        plt.show()


# 选择一张测试图像进行可视化
sample_idx = np.random.randint(0, len(test_images))
sample_image = test_images[sample_idx]
sample_label = np.argmax(test_labels[sample_idx])

print(f"可视化数字 {sample_label} 的特征图:")

# 显示原图
plt.figure(figsize=(3, 3))
plt.imshow(sample_image.squeeze(), cmap='gray')
plt.title(f'Original Image (Label: {sample_label})')
plt.axis('off')
plt.show()

# 显示特征图
visualize_feature_maps(model, sample_image)

## 7. 总结

### 实验结果

本实验实现了一个简洁而高效的CNN模型：
- 模型参数量：约93K
- 测试集准确率：约99%
- 训练时间：约2分钟（5个epoch）

### 核心知识点回顾

1. **卷积层**：通过滤波器提取局部特征，参数共享减少模型复杂度
2. **池化层**：下采样实现特征降维和平移不变性
3. **全连接层**：将空间特征映射到分类空间
4. **激活函数**：ReLU解决梯度消失，Softmax产生概率分布

### 进阶方向

1. **数据增强**：旋转、平移、缩放等增强泛化能力
2. **正则化**：Dropout、L2正则化防止过拟合
3. **批归一化**：加速训练收敛
4. **更深的网络**：VGG、ResNet等经典架构

In [ ]:
# 最终结果汇总
print("=" * 50)
print("实验结果汇总")
print("=" * 50)
print(f"模型名称: SimpleCNN")
print(f"总参数量: {model.count_params():,}")
print(f"训练轮次: {EPOCHS}")
print(f"最终训练准确率: {history.history['accuracy'][-1]:.4f}")
print(f"最终验证准确率: {history.history['val_accuracy'][-1]:.4f}")
print(f"测试集准确率: {test_accuracy:.4f}")
print("=" * 50)